In [4]:
import shutil
import numpy as np
import os

In [27]:
imagenet_dir = "/Users/ninawiedemann/Desktop/Projects/data_pocovid/tiny-imagenet-200/train"
nerve_segment_dir = "/Users/ninawiedemann/Desktop/Projects/data_pocovid/test"
out_dir = "/Users/ninawiedemann/Desktop/Projects/data_pocovid/uniform_class_imagenet"
out_dir_nerves = "/Users/ninawiedemann/Desktop/Projects/data_pocovid/uniform_class_nerves"

### Copy data from imagenet

In [21]:
n_per_class = 3
for img_class in os.listdir(imagenet_dir):
    if img_class[0]==".":
        continue
    class_path = os.path.join(imagenet_dir, img_class, "images")
    # print(class_path)
    class_files = np.array(os.listdir(class_path))
    inds = np.random.permutation(len(class_files))[:n_per_class]
    for chosen_file in class_files[inds]:
        # print(os.path.join(class_path, chosen_file))
        shutil.copy(os.path.join(class_path, chosen_file), out_dir)

### copy data from nerve segmentation

In [11]:
from PIL import Image

In [22]:
n_nerves = 400
all_tifs = np.array(os.listdir(nerve_segment_dir))
inds = np.random.permutation(len(all_tifs))[:n_nerves]
chosen_files = all_tifs[inds]

In [23]:
for fn in chosen_files:
    infile = os.path.join(nerve_segment_dir, fn)
    if fn[-3:] == "tif":
        outfile = os.path.join(out_dir, fn[:-3] + "jpeg")
        im = Image.open(infile)
        out = im.convert("RGB")
        out.save(outfile, "JPEG", quality=90)

### Add data to cross validation split

In [31]:
SPLIT = 5

In [28]:
files_imagenet = os.listdir(out_dir)
files_nerves = os.listdir(out_dir_nerves)

In [33]:
crossval_dir = "/Users/ninawiedemann/Desktop/Projects/data_pocovid/cross_validation"

In [40]:
nr_each = len(files_nerves)//SPLIT
for i in range(SPLIT):
    fold_dir = os.path.join(crossval_dir, "split"+str(i), "uninformative")
    if not os.path.exists(fold_dir):
        os.makedirs(fold_dir)
    copy_files_imagenet = files_imagenet[i*nr_each:(i+1)*nr_each]
    copy_files_nerves = files_nerves[i*nr_each:(i+1)*nr_each]
    #print(len(copy_files))
    for c in copy_files_imagenet:
        shutil.copy(os.path.join(out_dir, c), fold_dir)
    for c in copy_files_nerves:
        shutil.copy(os.path.join(out_dir_nerves, c), fold_dir)